In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

In [2]:
os.path.join('data/train.csv')

'data/train.csv'

In [3]:
df = pd.read_csv('data/train.csv')

In [4]:
df.head()

id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0

In [5]:
df.iloc[6]['comment_text']

'COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK'

In [6]:
df[df.toxic==1].head()

id                                       comment_text  \
6   0002bcb3da6cb337       COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK   
12  0005c987bdfc9d4b  Hey... what is it..\n@ | talk .\nWhat is it......   
16  0007e25b2121310b  Bye! \n\nDon't look, come or think of comming ...   
42  001810bf8c45bf5f  You are gay or antisemmitian? \n\nArchangel WH...   
43  00190820581d90ce           FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!   

    toxic  severe_toxic  obscene  threat  insult  identity_hate  
6       1             1        1       0       1              0  
12      1             0        0       0       0              0  
16      1             0        0       0       0              0  
42      1             0        1       0       1              1  
43      1             0        1       0       1              0

In [7]:
df[df.columns[2:]].iloc[6]

toxic            1
severe_toxic     1
obscene          1
threat           0
insult           1
identity_hate    0
Name: 6, dtype: int64

In [8]:
df.shape

(159571, 8)

# 1. Preprocess

In [9]:
from tensorflow.keras.layers import TextVectorization

In [10]:
X = df.comment_text

In [11]:
y = df[df.columns[2:]].values

In [14]:
MAX_WORDS = 200000

In [16]:
vectorizer = TextVectorization(max_tokens = MAX_WORDS,output_sequence_length = 1800,
                              output_mode = 'int')

Metal device set to: Apple M1 Pro


In [17]:
vectorizer.adapt(X.values)

In [39]:
vectorizer.get_vocabulary() #-----> Returns the current vocabulary of the layer.


['',
 '[UNK]',
 'the',
 'to',
 'of',
 'and',
 'a',
 'you',
 'i',
 'is',
 'that',
 'in',
 'it',
 'for',
 'this',
 'not',
 'on',
 'be',
 'as',
 'have',
 'are',
 'your',
 'with',
 'if',
 'article',
 'was',
 'or',
 'but',
 'page',
 'my',
 'an',
 'from',
 'by',
 'do',
 'at',
 'about',
 'me',
 'so',
 'wikipedia',
 'can',
 'what',
 'there',
 'all',
 'has',
 'will',
 'talk',
 'please',
 'would',
 'its',
 'no',
 'one',
 'just',
 'like',
 'they',
 'he',
 'dont',
 'which',
 'any',
 'been',
 'should',
 'more',
 'we',
 'some',
 'other',
 'who',
 'see',
 'here',
 'also',
 'his',
 'think',
 'im',
 'because',
 'know',
 'how',
 'am',
 'people',
 'why',
 'edit',
 'articles',
 'only',
 'out',
 'up',
 'when',
 'were',
 'use',
 'then',
 'may',
 'time',
 'did',
 'them',
 'now',
 'being',
 'their',
 'than',
 'thanks',
 'even',
 'get',
 'make',
 'good',
 'had',
 'very',
 'information',
 'does',
 'could',
 'well',
 'want',
 'such',
 'sources',
 'way',
 'name',
 'these',
 'deletion',
 'pages',
 'first',
 'help'

In [40]:
vectorizer_text = vectorizer(X.values)

In [41]:
len(X)

159571

In [43]:
vectorizer_text #--> 1800 represendts output sequence length

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

# Remember MCSHBAP
* M ----> MAP
* C ----> CACHE
* SH----> SHUFFLE
* BA----> BATCH
* P ----> PREFETCH

In [53]:
dataset = tf.data.Dataset.from_tensor_slices((vectorizer_text,y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # ---->It prevents the bottlenecks

In [54]:
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 1800), dtype=tf.int64, name=None), TensorSpec(shape=(None, 6), dtype=tf.int64, name=None))>

In [56]:
batch_X,batch_y = dataset.as_numpy_iterator().next()

In [58]:
batch_X,batch_y.shape

(array([[5495, 1220,   73, ...,    0,    0,    0],
        [   5,  390,   42, ...,    0,    0,    0],
        [1885,  468,  179, ...,    0,    0,    0],
        ...,
        [ 901, 1048,  880, ...,    0,    0,    0],
        [   8,   19,  569, ...,    0,    0,    0],
        [ 165,    6, 1024, ...,    0,    0,    0]]),
 (16, 6))

In [63]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip((int(len(dataset)*.7))).take(int(len(dataset)* .2))
test = dataset.skip((int(len(dataset)*.9))).take(int(len(dataset)*.1))

In [65]:
len(test)

997

In [66]:
train.as_numpy_iterator().next()

(array([[  323,    10,    14, ...,     0,     0,     0],
        [ 6613,    16,  1484, ...,     0,     0,     0],
        [    2, 78966,    11, ...,     0,     0,     0],
        ...,
        [  485,  2359,    49, ...,     0,     0,     0],
        [  669,  4541,   244, ...,     0,     0,     0],
        [    8,   954,     3, ...,     0,     0,     0]]),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]))

# 2. Creating Sequential Model